## Model Describer Meetup Tutorial

In this notebook, we will be doing some brief EDA of the [bicycle trip dataset](https://www.kaggle.com/pronto/cycle-share-dataset/home). This is data from the Pronto Cycle Share system which consists of 500 bikes and 54 stations located in Seattle. 

The key question for this tutorial will be whether or not there are noticeable differences in trip duration by gender and by user age. We will not be controlling for location information (a known deficit of this tutorial). 

In this tutorial, we will be covering the following:

* [Data prep](#prep)
* [Exploratory data analysis](#eda)
* [Build neural network model](#neural)
* [Model Describer Evaluation](#mdesc_regression)
* [Model Describer Sensitivity](#mdesc_sensitivity_regression)
* [Additional thoughts](#thoughts)

In [1]:
import os
from datetime import datetime
import warnings

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import keras
import pandas as pd

from mdesc.models import (Eval, ClassifierEval, Sensitivity, ClassifierSensitivity)

/Users/jasonlewris/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [2]:
# initialize plotly notebook_mode
init_notebook_mode(connected=True)

# filter out warning messages
warnings.filterwarnings('ignore')

#### Data Prep <a id='prep' />

Read in data and perform basic data manipulations

In [3]:
# read in the trip and weather data. We will not be using the station data, but there are a number of ways we could use
# this if we took advantage of the location information. 
base_path = '/Users/jasonlewris/Desktop/cycle-share-dataset/'
# base_path = r'C:\Users\jlewris\Desktop\BikeData'
trip = pd.read_csv(os.path.join(base_path, 'trip.csv'), error_bad_lines=False)
weather = pd.read_csv(os.path.join(base_path, 'weather.csv'))

b'Skipping line 50794: expected 12 fields, saw 20\n'


In [4]:
# for the purposes of this tutorial, we are going to drop rows without demographic data
trip = trip.loc[trip['gender'].notnull()]

In [5]:
# some basic preprocessing functions that should be pulled out into a utility file

def convert_date(dte):
    """
    convert string date into datetime object
    
    Parameters
    ----------
    dte - string
          datetime string object
          
    Return 
    ----------
    datetime obj
        string input converted to datetime object
    """
    try:
        dte = datetime.strptime(dte, '%m/%d/%Y %H:%M')
    except ValueError:
        dte = datetime.strptime(dte, '%m/%d/%Y')
    return dte

def return_part_date(dte, part_of_date='month'):
    """
    Pull the part_of_date from input datetime object
    
    Parameters
    ----------
    dte - datetime object
          input datetime object
    
    part_of_date - str - ['month', 'day', 'year', 'hour', 'minute']
          
    Return 
    ----------
    part of date
        part of date, i.e. year, hour, etc. 
    """
    dte = convert_date(dte)
    return getattr(dte, part_of_date)

def is_weekday(dte):
    """
    return whether a dte is a weekday or not
    
    Parameters
    ----------
    dte - datetime object
          input datetime object
              
    Return 
    ----------
    binary flag
        1 if is weekday else 0 
    """
    dte = convert_date(dte)
    if dte.weekday() not in [5, 6]:
        return 1
    else:
        return 0
    
def map_hours(hr):
    """
    map input hour into bin
    
    Parameters
    ----------
    hr - int - required
          input hour
              
    Return 
    ----------
    string
        bin of hour (early_morning, morning_rush, mid_day, evening_rush, evening)
    """
    if hr < 6:
        return 'early_morning'
    elif hr >= 6 and hr < 10:
        return 'morning_rush'
    elif hr >= 10 and hr < 15:
        return 'mid_day'
    elif hr >= 15 and hr < 19:
        return 'evening_rush'
    else:
        return 'evening'

In [6]:
# pull relevant parts of date
trip['start_day'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='day'))
trip['start_year'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='year'))
trip['start_month'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='month'))
trip['start_hour'] = trip['starttime'].apply(lambda x: return_part_date(x, part_of_date='hour'))
weather['start_day'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='day'))
weather['start_year'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='year'))
weather['start_month'] = weather['Date'].apply(lambda x: return_part_date(x, part_of_date='month'))

# test if date is weekday or not
trip['weekday'] = trip['starttime'].apply(lambda x: is_weekday(x))

In [7]:
# pull out just the mean values for weather
weather_sub = weather[['Mean_Temperature_F', 'MeanDew_Point_F', 'Mean_Humidity', 
                      'Mean_Visibility_Miles', 'Mean_Wind_Speed_MPH', 'Precipitation_In', 
                      'start_day', 'start_year', 'start_month']]

In [8]:
trip = pd.merge(trip, weather_sub, on=['start_day', 'start_year', 'start_month'], how='left')

In [9]:
# drop end location information
trip = trip.drop(['to_station_name', 'to_station_id', 'from_station_name'], axis=1)

In [10]:
# get age of rider
trip['rider_age'] = trip['start_year'] - trip['birthyear']

### Basic Exploratory Data Analysis <a id='eda' />

Perform basic exploratory analysis of the bicycle data. Some preliminary analysis includes:

* Number of trips by weekend/weekday by hour
* Average trip duration by weekend/weekday by hour
* Average trip duration by gender
* Correlation heatmap of key variables of interest

In [11]:
# number of trips by day type and hour
numtrips_weekday = trip.groupby(['weekday', 'start_hour'])['trip_id'].nunique().reset_index(name='numTrip')

total_weekday_trips = trip.loc[trip['weekday'] == 1]['trip_id'].nunique()
total_weekend_trips = trip.loc[trip['weekday'] == 0]['trip_id'].nunique()

numtrips_weekday['percentTrips'] = numtrips_weekday.apply(lambda x: x['numTrip']/total_weekday_trips if x['weekday'] == 1 else x['numTrip']/total_weekend_trips, axis=1)

In [12]:
# get number of trips by hour of day

weekday = numtrips_weekday['weekday'] == 1
weekend = numtrips_weekday['weekday'] == 0

trace1 = go.Bar(
    x = numtrips_weekday.loc[weekday, 'start_hour'].tolist(),
    y = numtrips_weekday.loc[weekday, 'percentTrips'].tolist(), 
    name='Weekday Trips'
)

trace2 = go.Bar(
    x = numtrips_weekday.loc[weekend, 'start_hour'].tolist(),
    y = numtrips_weekday.loc[weekend, 'percentTrips'].tolist(), 
    name='Weekend Trips'
)

data = [trace1, trace2]

layout = go.Layout(barmode='group', 
                  title='Weekend vs. Weekday Trips by Hour',
                  xaxis=dict(title='Hour'), 
                  yaxis=dict(title='Percent of Trips'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [13]:
# average trip duration by day type by hour
trip_dist = trip.groupby(['weekday', 'start_hour'])['tripduration'].mean().reset_index(name='tripDist')

In [14]:
trace1 = go.Scatter(
    x = trip_dist.loc[weekday, 'start_hour'].tolist(),
    y = trip_dist.loc[weekday, 'tripDist'].tolist(), 
    name='Weekday Trips', 
    mode='lines'
)

trace2 = go.Scatter(
    x = trip_dist.loc[weekend, 'start_hour'].tolist(),
    y = trip_dist.loc[weekend, 'tripDist'].tolist(), 
    name='Weekend Trips', 
    mode='lines'
)

data = [trace1, trace2]

layout = go.Layout(
                  title='Weekend vs. Weekday Trip Duration by Hour',
                  xaxis=dict(title='Hour'), 
                  yaxis=dict(title='Trip Duration (seconds)'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [15]:
# we can clearly see peaks during the weekday during rush hour. Lets map these to reduce our 
# total feature exposure

trip['start_hour'] = trip['start_hour'].apply(lambda x: map_hours(x))

In [16]:
tripgender = trip.groupby('gender')['tripduration'].mean().reset_index(name='tripDist')

data = [go.Bar(
    x=tripgender['gender'].tolist(),
    y=tripgender['tripDist'].tolist()
)]

layout = go.Layout(
                  title='Average Trip Duration (seconds) by Gender',
                  xaxis=dict(title='Gender'), 
                  yaxis=dict(title='Trip Duration (seconds)'))

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [17]:
corr = trip[['rider_age', 'Mean_Temperature_F', 'Mean_Humidity', 
             'tripduration', 'weekday']].corr()

trace = go.Heatmap(z=corr.values.tolist(), 
                   x=corr.index.tolist(),
                   y=corr.columns.tolist())

layout = go.Layout(
                  title='Correlation Heatmap',
    )

data = [trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

### Build Neural Network Model <a id='neural' /a>

Build out example neural network in keras to predict trip duration

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [19]:
# for the purposes of this tutorial, we are dropping the gender category other due to lack of observations. 
# more time could be spent doing a better sampling method for training to include sparse categories

# drop certain columns 
trip_model = trip.drop(['stoptime', 'bikeid', 'trip_id', 
                       'starttime', 'birthyear', 'start_day'], axis=1)



X = trip_model.loc[:, trip_model.columns != 'tripduration']
y = trip_model.loc[:, 'tripduration'].values.reshape((X.shape[0], 1))

In [20]:
# fill in na values
# X['rider_age'] = X['rider_age'].fillna(X['rider_age'].mean())
# X.fillna(X.mean(), inplace=True)

In [21]:
# map hours
X['weekday'] = X['weekday'].replace({0: 'weekend',
                                    1: 'weekday'})

In [22]:
# pull out categories of interest
categories = ['from_station_id', 'usertype', 
              'gender', 'start_year', 
              'start_month', 'start_hour', 'weekday']

continuous = [col for col in X.columns.tolist() if col not in categories]

In [23]:
# create binary values for categories 
X_cat = pd.get_dummies(X[categories], columns=categories)

X_cat_cols = X_cat.columns.tolist()

In [24]:
from sklearn.preprocessing import normalize
sc = StandardScaler()


# for col in continuous:
#    X[col] = X[col].fillna(X[col].mean())

X_cont = normalize(X[continuous])# sc.fit_transform(X[continuous])



In [25]:
X = np.hstack((X_cat, X_cont))

feature_names = X_cat_cols + continuous

In [26]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(X, y,trip, 
                                                   test_size=0.1, 
                                                   random_state=42)

In [27]:
# create a relatively simple neural network for regression purposes. 

model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu', 
               kernel_initializer='normal'))
model.add(Dropout(0.1))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))

# slow learning rate - continued drop in validation loss gives evidence that
# many more epochs might be sufficient - also installing tensorflow with GPU backend
# would dramatically speed up training
adam = Adam(lr=0.0005)

model.compile(loss='mae', optimizer=adam)
model.fit(X_train, y_train, epochs=500, verbose=1, batch_size=512, 
         validation_split=0.2)

Train on 130720 samples, validate on 32681 samples
Epoch 1/500
130720/130720 [==============================] - 2s 12us/step - loss: 369.2912 - val_loss: 269.9443
Epoch 2/500
130720/130720 [==============================] - 1s 10us/step - loss: 265.3493 - val_loss: 265.2714
Epoch 3/500
130720/130720 [==============================] - 1s 10us/step - loss: 262.8883 - val_loss: 264.2550
Epoch 4/500
130720/130720 [==============================] - 1s 11us/step - loss: 262.0109 - val_loss: 263.3195
Epoch 5/500
130720/130720 [==============================] - 1s 10us/step - loss: 261.0133 - val_loss: 262.1875
Epoch 6/500
130720/130720 [==============================] - 1s 10us/step - loss: 260.1779 - val_loss: 261.2826
Epoch 7/500
130720/130720 [==============================] - 1s 10us/step - loss: 259.2535 - val_loss: 260.0179
Epoch 8/500
130720/130720 [==============================] - 1s 10us/step - loss: 258.4504 - val_loss: 259.0641
Epoch 9/500
130720/130720 [==========================

130720/130720 [==============================] - 1s 11us/step - loss: 245.3233 - val_loss: 246.3045
Epoch 74/500
130720/130720 [==============================] - 1s 11us/step - loss: 245.2606 - val_loss: 245.9892
Epoch 75/500
130720/130720 [==============================] - 1s 11us/step - loss: 245.2083 - val_loss: 246.0283
Epoch 76/500
130720/130720 [==============================] - 1s 10us/step - loss: 244.8942 - val_loss: 245.7963
Epoch 77/500
130720/130720 [==============================] - 1s 11us/step - loss: 244.6040 - val_loss: 246.0265
Epoch 78/500
130720/130720 [==============================] - 1s 11us/step - loss: 244.5350 - val_loss: 245.9575
Epoch 79/500
130720/130720 [==============================] - 1s 11us/step - loss: 244.4313 - val_loss: 245.7859
Epoch 80/500
130720/130720 [==============================] - 1s 11us/step - loss: 244.3916 - val_loss: 245.3272
Epoch 81/500
130720/130720 [==============================] - 1s 11us/step - loss: 243.8802 - val_loss: 245.4

130720/130720 [==============================] - 1s 10us/step - loss: 236.5078 - val_loss: 239.7308
Epoch 146/500
130720/130720 [==============================] - 1s 10us/step - loss: 236.4234 - val_loss: 240.0269
Epoch 147/500
130720/130720 [==============================] - 1s 11us/step - loss: 236.3102 - val_loss: 239.8497
Epoch 148/500
130720/130720 [==============================] - 1s 11us/step - loss: 236.2073 - val_loss: 239.9363
Epoch 149/500
130720/130720 [==============================] - 1s 11us/step - loss: 236.4340 - val_loss: 239.9789
Epoch 150/500
130720/130720 [==============================] - 1s 10us/step - loss: 236.3886 - val_loss: 241.1832
Epoch 151/500
130720/130720 [==============================] - 1s 10us/step - loss: 236.1883 - val_loss: 239.0674
Epoch 152/500
130720/130720 [==============================] - 1s 11us/step - loss: 235.9632 - val_loss: 239.4424
Epoch 153/500
130720/130720 [==============================] - 1s 11us/step - loss: 235.8653 - val_los

Epoch 217/500
130720/130720 [==============================] - 1s 10us/step - loss: 230.9754 - val_loss: 236.1154
Epoch 218/500
130720/130720 [==============================] - 1s 11us/step - loss: 230.7934 - val_loss: 236.6357
Epoch 219/500
130720/130720 [==============================] - 1s 10us/step - loss: 230.7129 - val_loss: 235.9499
Epoch 220/500
130720/130720 [==============================] - 1s 11us/step - loss: 230.6976 - val_loss: 236.6176
Epoch 221/500
130720/130720 [==============================] - 1s 11us/step - loss: 230.8643 - val_loss: 236.4511
Epoch 222/500
130720/130720 [==============================] - 1s 10us/step - loss: 230.4942 - val_loss: 236.9905
Epoch 223/500
130720/130720 [==============================] - 1s 11us/step - loss: 230.5567 - val_loss: 236.5787
Epoch 224/500
130720/130720 [==============================] - 1s 11us/step - loss: 230.3788 - val_loss: 236.1722
Epoch 225/500
130720/130720 [==============================] - 1s 11us/step - loss: 230.

130720/130720 [==============================] - 1s 11us/step - loss: 227.0507 - val_loss: 235.3286
Epoch 289/500
130720/130720 [==============================] - 1s 10us/step - loss: 226.6956 - val_loss: 234.0479
Epoch 290/500
130720/130720 [==============================] - 1s 11us/step - loss: 227.0438 - val_loss: 234.3446
Epoch 291/500
130720/130720 [==============================] - 1s 10us/step - loss: 227.0725 - val_loss: 233.9627
Epoch 292/500
130720/130720 [==============================] - 1s 11us/step - loss: 226.5909 - val_loss: 234.6574
Epoch 293/500
130720/130720 [==============================] - 1s 11us/step - loss: 226.7014 - val_loss: 233.8727
Epoch 294/500
130720/130720 [==============================] - 1s 11us/step - loss: 226.5321 - val_loss: 234.4225
Epoch 295/500
130720/130720 [==============================] - 1s 11us/step - loss: 226.4935 - val_loss: 233.8331
Epoch 296/500
130720/130720 [==============================] - 1s 11us/step - loss: 226.9719 - val_los

Epoch 360/500
130720/130720 [==============================] - 1s 11us/step - loss: 223.8713 - val_loss: 232.6125
Epoch 361/500
130720/130720 [==============================] - 1s 11us/step - loss: 223.7337 - val_loss: 232.4106
Epoch 362/500
130720/130720 [==============================] - 1s 11us/step - loss: 224.0818 - val_loss: 233.0832
Epoch 363/500
130720/130720 [==============================] - 1s 11us/step - loss: 224.0137 - val_loss: 232.5650
Epoch 364/500
130720/130720 [==============================] - 1s 11us/step - loss: 223.7728 - val_loss: 232.4744
Epoch 365/500
130720/130720 [==============================] - 1s 11us/step - loss: 223.7387 - val_loss: 232.8345
Epoch 366/500
130720/130720 [==============================] - 1s 11us/step - loss: 223.6665 - val_loss: 232.4759
Epoch 367/500
130720/130720 [==============================] - 1s 11us/step - loss: 223.6934 - val_loss: 232.4146
Epoch 368/500
130720/130720 [==============================] - 1s 11us/step - loss: 223.

130720/130720 [==============================] - 1s 10us/step - loss: 221.3120 - val_loss: 231.8121
Epoch 432/500
130720/130720 [==============================] - 1s 10us/step - loss: 221.2594 - val_loss: 232.0997
Epoch 433/500
130720/130720 [==============================] - 1s 10us/step - loss: 221.2963 - val_loss: 231.9685
Epoch 434/500
130720/130720 [==============================] - 1s 10us/step - loss: 221.5742 - val_loss: 232.2199
Epoch 435/500
130720/130720 [==============================] - 1s 10us/step - loss: 221.3710 - val_loss: 231.4932
Epoch 436/500
130720/130720 [==============================] - 1s 11us/step - loss: 221.5893 - val_loss: 231.3920
Epoch 437/500
130720/130720 [==============================] - 1s 10us/step - loss: 221.3738 - val_loss: 231.7164
Epoch 438/500
130720/130720 [==============================] - 1s 10us/step - loss: 221.3726 - val_loss: 231.5125
Epoch 439/500
130720/130720 [==============================] - 1s 10us/step - loss: 221.3271 - val_los

### Model Describer Evaluation <a id='mdesc_regression' />

Fitting model describer evaluation function is easy. Simply provide the models prediction function directly, 
and feed in X, and y. The remainder of the parameters are for beuatification of outputs. 

The most straight forward instantiation is with no beutification and no groupby dataframe:

```python
from mdesc.models import Eval

RE = Eval(prediction_fn=model.predict)

res = RE.fit_transform(X=X, y=y)
```

In [28]:
# capture relevant groupby values
groupby_df = group_test[['gender',  'usertype', 'start_hour']]

In [29]:
# fitting model describer evaluation function is easy. Simply provide the models prediction function directly, 
# and feed in X, and y. The remainder of the parameters are for beuatification of outputs
RE = Eval(prediction_fn=model.predict, target_names='transit_time', 
                feature_names=feature_names)

res = RE.fit_transform(X=X_test, y=y_test.flatten(), 
                       groupby_df=groupby_df)

In [31]:
RE.data_set.viz_now(groupby_name='gender')

### Model Describer Evaluation <a id='mdesc_sensitivity_regression' />

Model describer sensitivity is also easy to instantiate. The key difference between sensitivity and eval is the ability to set how many standard deviations you want to synthesize the data. That can be set using the std parameter. 

Without any naming of columns or y levels, you can instantiate Sensitivity with the following:

```python
from mdesc.models import Sensitivity

RE = Sensitivity(prediction_fn=model.predict)

res = RE.fit_transform(X=X, y=y)
```

In [34]:
RE = Sensitivity(prediction_fn=model.predict, target_names='transit_time (seconds)', 
                feature_names=feature_names, std=0.5)

res = RE.fit_transform(X=X_test[0:10000], y=y_test[0:10000].flatten(), 
                       groupby_df=groupby_df[0:10000])

In [35]:
RE.data_set.viz_now(groupby_name='gender')